In [2]:
import numpy as np
from bisect import bisect_right as BSR, bisect_left as BSL 

def generateSymetryBase():
    s3d2 = np.sqrt(3)/2
    return {
        "c":{
            "000":np.array([[-1,0,0],[0,-1,0],[0,0,-1]])
        },
        "m":{
            "100":np.array([[-1,0,0],[0,1,0],[0,0,1]]),
            "010":np.array([[1,0,0],[0,-1,0],[0,0,1]]),
            "001":np.array([[1,0,0],[0,1,0],[0,0,-1]]),

            "110":np.array([[0,-1,0],[-1,0,0],[0,0,1]]),            
            "011":np.array([[1,0,0],[0,0,-1],[0,-1,0]]),
            "101":np.array([[0,0,-1],[0,1,0],[-1,0,0]]),
            
            "-101":np.array([[0,0,1],[0,1,0],[1,0,0]]),
            "-110":np.array([[0,1,0],[1,0,0],[0,0,1]]),
            "0-11":np.array([[1,0,0],[0,0,1],[0,1,0]])
        },
        "2":{
            "100":np.array([[1,0,0],[0,-1,0],[0,0,-1]]),
            "010":np.array([[-1,0,0],[0,1,0],[0,0,-1]]),
            "001":np.array([[-1,0,0],[0,-1,0],[0,0,1]]),

            "110":np.array([[0,1,0],[1,0,0],[0,0,-1]]),
            "101":np.array([[1,0,0],[0,-1,0],[0,0,1]]),
            "011":np.array([[-1,0,0],[0,0,1],[0,1,0]]),

            "-110":np.array([[0,-1,0],[-1,0,0],[0,0,-1]]),
            "-101":np.array([[0,0,-1],[0,-1,0],[-1,0,0]]),
            "0-11":np.array([[-1,0,0],[0,0,-1],[0,-1,0]])
        },
        "3":{
#             "001":np.array([[-0.5,-s3d2,0],[s3d2,-0.5,0],[0,0,1]]),
            "111":np.array([[0,0,1],[1,0,0],[0,1,0]]),
            "-111":np.array([[0,-1,0],[0,0,1],[-1,0,0]]),
            "1-11":np.array([[0,-1,0],[0,0,-1],[1,0,0]]),
            "11-1":np.array([[0,1,0],[0,0,-1],[-1,0,0]])
        },
        "-3":{
#             "001":np.array([[0.5,s3d2,0],[-s3d2,0.5,0],[0,0,-1]]),
            "111":np.array([[0,0,-1],[-1,0,0],[0,-1,0]]),
            "-111":np.array([[0,1,0],[0,0,-1],[1,0,0]]),
            "1-11":np.array([[0,1,0],[0,0,1],[-1,0,0]]),
            "11-1":np.array([[0,-1,0],[0,0,1],[1,0,0]])        
        },
        "4":{
            "100":np.array([[1,0,0],[0,0,-1],[0,1,0]]),
            "010":np.array([[0,0,1],[0,1,0],[-1,0,0]]),
            "001":np.array([[0,-1,0],[1,0,0],[0,0,1]])
        },
        "-4":{
            "100":np.array([[-1,0,0],[0,0,1],[0,-1,0]]),
            "010":np.array([[0,0,-1],[0,-1,0],[1,0,0]]),
            "001":np.array([[0,1,0],[-1,0,0],[0,0,-1]])
        }#,
#         "6":{
#             "001":np.array([[0.5,-s3d2,0],[s3d2,0.5,0],[0,0,1]])
#         },
#         "-6":{
#             "001":np.array([[-0.5,s3d2,0],[-s3d2,-0.5,0],[0,0,-1]])
#         }
    }

In [3]:
# def listadous(macierz, punktprzes):
#     punkt = np.matmul(macierz,punktprzes)  # np.matmul
#     punkt = np.around(punkt,6)
#     while not np.allclose(punkt, punktprzes):   # porownajPunkty
#         yield punkt
#         punkt = np.matmul(macierz,punkt)   
#         punkt = np.around(punkt,6) 
        
def listadous(macierz, punktprzes):
    punkt = np.matmul(macierz,punktprzes)  # np.matmul
    while not porownajPunkty(punkt, punktprzes):   # porownajPunkty
        yield punkt
        punkt = np.matmul(macierz,punkt)        
        
# def findAntiPoints(mylist,zbior2):
#     for punktprzek in mylist:
#         indx = findindex(punktprzek,zbior2)               
#         if indx + 1 and porownajPunkty(zbior2[:,indx],punktprzek): #porownajPunkty()
#             return False
#     return True    

def supercell(cellorg, size):
    if size == 1:
        return np.unique(cellorg,axis=0)*2-1
    elif size < 1:
        return None
    cell = cellorg.copy()
    for os in range(3):    
        subcell = cell.copy()[cell.T[os]!=0]
        listcell = subcell.copy()
        for _ in range(1,size):        
            subcell[:,os] += 1
            listcell = np.append(listcell,subcell,axis =0)   
        cell = np.append(cell,listcell,axis =0)   
    cell = np.unique(cell,axis = 0)/(size/2)-1
    return np.around(cell,6)

def myswitch(nr):
    mydict = {
        "c": 1,
        "m": 1,
        "2": 1,
        "3": 2,
        "-3": 5,
        "4": 3,
        "-4": 3,
        "6": 5,
        "-6": 5
    }
    return mydict[nr]

def makeCellWithVacancies(cell,indexes):
    indexes = set(indexes)
    cellVac = []
    Vac = []
    for nr, p in enumerate(cell):
        if nr not in indexes:
            cellVac.append(p)
        else:
            Vac.append(p)
    return np.array(cellVac), np.array(Vac)

def listadous(macierz, punktprzes):
    punkt = np.matmul(macierz,punktprzes)  # np.matmul
    punkt = np.around(punkt,6)
    while not porownajPunkty(punkt, punktprzes):   # porownajPunkty
        yield punkt
        punkt = np.matmul(macierz,punkt)   
        punkt = np.around(punkt,6) 

def usunkoor(koorZEW, oski):
    koorWEW = koorZEW.copy()
    for os in oski:
        j = 1
        while j < len(koorWEW):            
            for punktdous in listadous(os, koorWEW[j]):                
                i = findindex(punktdous, koorWEW.T[:,j:])                
                if i + 1 and porownajPunkty(koorWEW[i+j],punktdous):
                    koorWEW = np.delete(koorWEW, i+j, 0)
            j += 1
    return koorWEW

def odlegloscmiedzypunktami(px1,py1,pz1,px2,py2,pz2):
    return np.round(np.sqrt((px1-px2)**2+(py1-py2)**2+(pz1-pz2)**2),4)

def printsym():
    for Mainkey in Matrixes.keys():
        for Subkey in Matrixes[Mainkey].keys():
            print((Mainkey,Subkey))
            print(Matrixes[Mainkey][Subkey],end='\n\n')
        print()
        
def makelist():
    Matrixes = generateSymetryBase()
    mylist = []
    for Mainkey in Matrixes.keys():
        for Subkey in Matrixes[Mainkey].keys():
            mylist.append((Mainkey,Subkey))
    return mylist[::-1]

def porownajPunkty(p1,p2):
    for a, b in zip(p1,p2):
        if a!=b:
            return False
    return True

def porownajMacierze(m1,m2):
    for a, b in zip(m1,m2):
        if porownajPunkty(a,b):
            return False
    return True

def findindex(searched, points):         
    indexL, indexR = 0, len(points[0])        
    for xyz in range(len(lista)): 
        numR = BSR(points[xyz][indexL: indexR], searched[xyz])
        numL = BSL(points[xyz][indexL: indexR], searched[xyz])        
        if numL < numR:
            indexR = indexL + numR
            indexL += numL
        elif numL == numR:
            indexL += numL
            if indexL != len(points[0]):
                return indexL
            return -1
        else:
            return -1        
    return indexL 

def findAntiPoints(listofvac, allpoints2):
    for vacsym in listofvac:
        indx = findindex(vacsym,allpoints2)               
        if indx + 1 and porownajPunkty(allpoints2[:,indx],vacsym): #porownajPunkty()
            return False
    return True

def findAntiSym_InnerLoop(Matrix, allpoints, vacancies):
    for vac in vacancies:
        listofvacsym = listadous(Matrix,vac)
        if not findAntiPoints(listofvacsym,allpoints.T):
            return False
    return True

def findAntiSym_MOD(matrixes, allpoints, vacancies):
    allsymmerties = makelist()
    if vacancies.shape == (3,):
        vacancies = np.array([vacancies])
    possymmerties = []    
    for el0, el1 in mylist:
        if findAntiSym_InnerLoop(matrixes[el0][el1], allpoints, vacancies):
            possymmerties.append((el0, el1))           
    return possymmerties 

def findPoints_MIXED(listofps,allpoints2,Anti = False):
    for ps in listofps:
        indx = findindex(ps,allpoints2)  
        cond = indx + 1 and porownajPunkty(allpoints2[:,indx],ps) #porownajPunkty() 
        if Anti:
            if cond:             
                return False
        else:
            if not cond:        
                return False
    return True 

def findSym_Base_mod2_innerLoop(Matrix, allpoints): #to samo co w anty?
    for p in allpoints:
        listofps = listadous(Matrix,p)          
        if not findPoints_MIXED(listofps,allpoints.T):
            return False
    return True

def findSym_Base_mod2(matrixes, allpoints, possym=makelist(),):
    symmerties = []
    for el0, el1 in possym:                
        if findSym_Base_mod2_innerLoop(matrixes[el0][el1], allpoints):
            symmerties.append((el0, el1))
    return symmerties

In [7]:
def abcd(scell,sumVac):
    mylist = []
    for indexes in itrtls_combinations(range(len(scell)),sumVac):   
#         print(indexes)
        scellvac, vacancies = makeCellWithVacancies(scell,indexes)
        mozliwosci = findAntiSym_MOD(Matrixes, scellvac, vacancies)
        SYM = findSym_Base_mod2(Matrixes, scellvac, mozliwosci)
        mylist.append(SYM)
    return mylist

def abcdmod(scell,sumVac):
    mylist = []
    for indexes in itrtls_combinations(range(len(scell)),sumVac):   
#         print(indexes)
        scellvac, vacancies = makeCellWithVacancies(scell,indexes)
        mozliwosci = findAntiSym_MOD(Matrixes, scellvac, vacancies)
        SYM = findSym_Base_mod2(Matrixes, scellvac, mozliwosci)
        mylist.append(SYM)
    return mylist

In [5]:
from Profiling import profile
# from MMfunc import *
from cifParse import *
from itertools import combinations as itrtls_combinations

filename = 'ZnS-Sfaleryt.cif'
CELL = getCellFromFile(filename)
SUPERCELL = supercell(CELL, size = 2)
print(len(SUPERCELL))
Matrixes = generateSymetryBase()

95


In [9]:
%time lol2 = abcd(SUPERCELL,2) #SC:4 VAC:2 Wall time:24min 27s

Wall time: 15.6 s


In [8]:
print(len(lol))
for el in sorted(lol):
    if not el == []:
        print(el)

192510
[('-4', '001'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-110'), ('m', '110')]
[('-4', '001'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-110'), ('m', '110')]
[('-4', '010'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-101'), ('m', '101')]
[('-4', '010'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '-101'), ('m', '101')]
[('-4', '100'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '0-11'), ('m', '011')]
[('-4', '100'), ('2', '001'), ('2', '010'), ('2', '100'), ('m', '0-11'), ('m', '011')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2', '001')]
[('2',

[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '

[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '-101')]
[('m', '

[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '

[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '-110')]
[('m', '

...

[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '011')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '10

[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '10

[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '101')]
[('m', '10

[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '11

[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '110')]
[('m', '11